In [1]:
import keras
import keras.layers as layers
import os
import numpy as np
import cv2
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
from keras.layers import Input, Dropout
from keras.applications import MobileNetV2
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Activation, Flatten, Dense
from keras.models import model_from_json

In [2]:
face_det = cv2.CascadeClassifier(r'C:\Users\mn_aj\Anaconda3\pkgs\libopencv-4.2.0-py37_3\Library\etc\haarcascades\haarcascade_frontalface_default.xml')
eyes_det = cv2.CascadeClassifier(r'C:\Users\mn_aj\Anaconda3\pkgs\libopencv-4.2.0-py37_3\Library\etc\haarcascades\haarcascade_eye.xml')

In [3]:
#build the Lenet model
model = keras.Sequential()
model.add(layers.Conv2D(filters = 6, kernel_size = (3,3), activation = 'relu', input_shape = (224,224,3)))
model.add(layers.AveragePooling2D())
model.add(layers.Conv2D(filters = 3, kernel_size=(3,3), activation = 'relu'))
model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(units = 184, activation = 'relu'))
model.add(layers.Dense(units = 84, activation = 'relu'))
model.add(layers.Dense(units = 2, activation = 'softmax'))

In [4]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [82]:
model.fit(train_batch, steps_per_epoch = 15, validation_data = validation_batch, validation_steps = 5, epochs = 20, verbose = 1)

Epoch 1/20
15/15 [==============================] - 8s 559ms/step - loss: 9.5917 - accuracy: 0.6133 - val_loss: 2.4569 - val_accuracy: 0.5217
Epoch 2/20
15/15 [==============================] - 7s 463ms/step - loss: 1.5341 - accuracy: 0.6200 - val_loss: 0.6200 - val_accuracy: 0.6190
Epoch 3/20
15/15 [==============================] - 7s 444ms/step - loss: 1.0269 - accuracy: 0.6533 - val_loss: 0.2997 - val_accuracy: 0.6667
Epoch 4/20
15/15 [==============================] - 7s 443ms/step - loss: 0.8116 - accuracy: 0.7063 - val_loss: 1.0190 - val_accuracy: 0.6957
Epoch 5/20
15/15 [==============================] - 7s 455ms/step - loss: 0.6331 - accuracy: 0.7733 - val_loss: 1.2053 - val_accuracy: 0.8571
Epoch 6/20
15/15 [==============================] - 7s 460ms/step - loss: 0.7002 - accuracy: 0.8000 - val_loss: 1.0502 - val_accuracy: 0.8571
Epoch 7/20
15/15 [==============================] - 7s 457ms/step - loss: 0.6801 - accuracy: 0.7667 - val_loss: 0.1867 - val_accuracy: 0.9565
Epoch 

In [5]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
modela = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

C:\Users\mn_aj\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [6]:
modela.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [7]:
modela.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
#Now create path for training dataset
train_path = 'train'
validation_path = 'validation'

In [9]:
# Image Data Generator
train_batch = ImageDataGenerator().flow_from_directory(train_path, target_size=(224, 224), classes = ['mask', 'unmask'], batch_size = 10)
validation_batch = ImageDataGenerator().flow_from_directory(validation_path, target_size=(224, 224), classes = ['mask', 'unmask'], batch_size = 5)


Found 1523 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [11]:
modela.fit(train_batch, steps_per_epoch = 15, validation_data = validation_batch, validation_steps = 5, epochs = 5, verbose = 1)

Epoch 1/5
15/15 [==============================] - 40s 3s/step - loss: 0.6833 - accuracy: 0.6667 - val_loss: 0.6317 - val_accuracy: 0.4348
Epoch 2/5
15/15 [==============================] - 23s 2s/step - loss: 0.3699 - accuracy: 0.8467 - val_loss: 1.3630 - val_accuracy: 0.3333
Epoch 3/5
15/15 [==============================] - 23s 2s/step - loss: 0.3003 - accuracy: 0.8867 - val_loss: 0.7412 - val_accuracy: 0.4762
Epoch 4/5
15/15 [==============================] - 23s 2s/step - loss: 0.2398 - accuracy: 0.8867 - val_loss: 1.1658 - val_accuracy: 0.4348
Epoch 5/5
15/15 [==============================] - 23s 2s/step - loss: 0.2199 - accuracy: 0.9000 - val_loss: 0.6660 - val_accuracy: 0.4286


In [10]:
def extract_feature_img(img):
    vgg16_feature_list = []
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feature_list.append(vgg16_feature_np.flatten())

    vgg16_feature_list_np = np.array(vgg16_feature_list)
    
    return vgg16_feature_list_np

In [11]:
directory = 'test_dataset'
for filename in os.listdir(directory):
    img = image.load_img(os.path.join(directory,filename), target_size=(224, 224))
   # test_vector = extract_feature_img(img)
   # labels = modela.predict(test_vector)
    np_image = np.array(img).astype('float32')/255
    np_image = cv2.resize(np_image, (224, 224))
    np_image = np.expand_dims(np_image, axis=0)
    labels = modela.predict(np_image)
    if labels[0][0]>labels[0][1]:
        print('mask')
    else:
        print('unmask')

unmask
unmask
unmask
unmask
unmask
mask
mask


In [14]:
#Save model
model_json = modela.to_json()
with open('modela.json', 'w') as json_file:
    json_file.write(model_json)


In [15]:
#Save weights
modela.save_weights("modela.h5")
print("Weights saved")

Weights saved


In [2]:
#Load Model
json_file = open('modela.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modela.h5")
print("Loaded model from disk")
 

Loaded model from disk


In [3]:
loaded_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [4]:
def face_detection(img, size= 0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_det.detectMultiScale(gray,1.3, 5)
    for (x,y,w,h) in faces:
        x = x - 60
        w = w + 100
        y = y - 60
        h = h + 100
    
        cv2.rectangle(img , (x,y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y: y+h, x: x+w]
        roi_img = img[y:y+h, x: x+w]
        cv2.imwrite('face.jpeg', roi_img)
        
        
        np_image = np.array(roi_img).astype('float32')/255
        np_image = cv2.resize(np_image, (224, 224))
        np_image = np.expand_dims(np_image, axis=0)
        labels = loaded_model.predict(np_image)
        #labels = modela.predict(np_image)
        if labels[0][0]>labels[0][1]:
            print('mask')
            cv2.putText(img, 'Mask Present',(x,y), 0, 0.5, (255,255,255),2)
        else:
            print('unmask')
            cv2.putText(img,"No mask", (x,y), 0, 0.5, (255,255,255),2)
        
        #resize = cv2.resize(roi_img,(224,224))
        #feature = extract_feature_img(resize)
        #labels = clf.predict(feature)
        #if labels == 1:
         #   print('Masked')
          #  cv2.putText(img, 'masked',(x,y), 0, 0.5, (255,255,255),2)
#         else:
#             print('Not masked')
#             cv2.putText(img,"not masked", (x,y), 0, 0.5, (255,255,255),2)
        
        eyes = eyes_det.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_img , (ex,ey), (ex+ew, ey+eh), (0, 0, 255), 2)
    return img

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 5)
while cap.isOpened():
    ret, im = cap.read()
    cv2.imshow('detect',face_detection(im))
    if cv2.waitKey (1) == 13: #13 is the Enter Key
        break
# When everything done,release the capture
cap.release()
cv2.destroyAllWindows()